<a href="https://colab.research.google.com/github/MarttB/CFD-project-report/blob/main/BumpGeom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# WITH BUMP
from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
# delta = np.pi / 28
# K = 1.3
# c = 0
# yp = 0
delta = np.pi / 15
K = 1
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c)) / ((1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2) * np.sin(x)/2

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_vals[-1]
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 25 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -3 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1

# .geo file creation:
with open("mesh.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1/2;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

    # Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

    # Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")
    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({3}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({4}) = {{{upper_point_end}, {upper_point_start}}};\n")

    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({5}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({16}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({19}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)


    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_10}) = {{{0}, {thick_bl}, 0, h}};\n")

    # Points for the B.L. of the convergent intake:
    #BL_1 = boundary_point_10 + 1
    # BL_conv_2 = BL_conv_1 + 1
    # BL_conv_3 = BL_conv_2 + 1
    # BL_conv_4 = BL_conv_3 + 1
    # BL_conv_5 = BL_conv_4 + 1
    # BL_conv_6 = BL_conv_5 + 1
    # BL_conv_7 = BL_conv_6 + 1

    #file.write(f"Point({BL_1}) = {{{np.argmax(x_vals)}, {thick_bl}, 0, h}};\n")
    #file.write(f"Line({100}) = {{{50}, {BL_1}, 0, h}};\n")
    #file.write(f"Transfinite Line({100}) = {n_bl} Using Progression {r};\n")
    # file.write(f"Point({BL_conv_4}) = {{{x_start_conv}, {y_lower_conv - thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_5}) = {{{x_start}, {y_start - thick_bl}, 0, h}};\n")
    # file.write(f"Point({BL_conv_6}) = {{{x_end}, {y_end - thick_bl}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    # file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{N_points_bump}, {boundary_point_6}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_7}, {new_point_end}}};\n")

    #file.write(f"Line({9}) = {{{new_point_end}, {upper_point_end}}};\n")

    file.write(f"Line({10}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({13}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({14}) = {{{boundary_point_4},{1} }};\n")  # Connect back to start of bump
    file.write(f"Line({20}) = {{{1},{boundary_point_10}}};\n")
    file.write(f"Line({21}) = {{{boundary_point_10},{boundary_point_5}}};\n")


    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{13}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{17}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{10}}} = {n_bl} Using Progression {r};\n")
    #file.write(f"Transfinite Line{{{8}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{19}}} = {10} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{15}}} = {100} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{2}}} = {140} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{16}}} = {10} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{18}}} = {150} Using Progression {r};\n")
    #file.write(f"Transfinite Line{{{5}}} = {80} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{6}}} = {80} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{14}}} = {300} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{20}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{21}}} = {300} Using Progression {1};\n")

    file.write("Line Loop(1) = {1, 2, 7, 8, 17, -3, -5, -19, -16, -6, -4, 10, 18, 11, 12, -21, -20};\n")
    file.write("Line Loop(2) = {14,20,21,13};\n")


    #Define the fluid domain
    file.write("Plane Surface(1) = {1};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(2) = {2};\n")
    file.write("Transfinite Surface{2} = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{1},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface{2};\n")


    # Boundary Layer:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {2};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {")
    file.write(f"{34}")
    file.write("};\n")
    file.write("Field[2].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 2;\n")

    file.write("Field[3] = BoundaryLayer;\n")
    file.write("Field[3].CurvesList = {19,5,3};\n")
    file.write("Field[3].Quads = 1;\n")
    file.write(f"Field[3].Ratio = {r};\n")
    file.write(f"Field[3].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[3].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[3].FanPointsList = {")
    file.write(f"{50}, {58}")
    file.write("};\n")
    file.write("Field[3].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 3;\n")

    file.write("Field[4] = BoundaryLayer;\n")
    file.write("Field[4].CurvesList = {16,6,4};\n")
    file.write("Field[4].Quads = 1;\n")
    file.write(f"Field[4].Ratio = {r};\n")
    file.write(f"Field[4].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[4].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[4].FanPointsList = {")
    file.write(f"{55}, {53}")
    file.write("};\n")
    file.write("Field[4].FanPointsSizesList = {")
    file.write(f"{5}")
    file.write("};\n")
    file.write("BoundaryLayer Field = 4;\n")


    # Physical Surface definition

    #file.write("Line Loop(20) = {14, 1, 2, 15, 7, 8, -17, -3, -5, -19, -16, -6, -4, 10, 18, 11, 12, 13};\n")
    #file.write("Plane Surface(10) = {20};\n")
    file.write("Physical Surface('VOLUME') = {1:2};\n")
    #file.write("Physical Surface('VOLUME2') = {2};\n")
    #file.write("Physical Surface('VOLUME3') = {4};\n")
    #file.write("Physical Surface('VOLUME4') = {5};\n")
    #file.write("Physical Surface('VOLUME5') = {6};\n")
    #file.write("Physical Surface('VOLUME6') = {7};\n")
    #file.write("Physical Surface('VOLUME7') = {8};\n")
    #file.write("Physical Surface('VOLUME8') = {9};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{12},{13}}};\n")

    file.write("Physical Line('lower_wall') = {")
    file.write(f"{14},")
    file.write("1, 2")
    file.write("};\n")

    #file.write(f"Physical Line('upper wall') = {{{11}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{3}, {4}, {6}, {16}, {19}, {5}")
    file.write("};\n")
    file.write(f"Physical Line('outlet') = {{{10}, {18},{11}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{7}, {8}, {17}}};\n")



from google.colab import files
files.download("mesh.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# WITHOUT BUMP
from os import write
import numpy as np
import matplotlib.pyplot as plt
import math

# Define bump parameters
# delta = np.pi / 28
# K = 1.3
# c = 0
# yp = 0
delta = np.pi / 15
K = 1
c = 0
yp = 0
# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c)) / ((1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump =50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 0 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.2 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)
x_start_bl = x_vals[index_line_bump - 10]

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = 4
line_length = End_line_param - x_start
x_end = x_vals[-1]
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 25 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -3 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl + 1


# .geo file creation:
with open("mesh_sbump.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01/2;\n") # Size of cells close to bump and convergent
    file.write("H = 1/2;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh


    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1
    center_convergent = circle_centre + 1

    # Write the convergent shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")


    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")


    y_upper_conv = y_start_conv - thick_conv2
    y_lower_conv = y_start_conv + thick_conv2 - thick_conv

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_upper_conv}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_lower_conv}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv + (x_start_up - x_start_conv)/200}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Computation of the radius of the circle of the convergent intake, given the center position and two points
    hhh = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx1 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy1 = y_upper_conv     #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio1 = math.sqrt((xxx1 - hhh)**2 + (yyy1 - kkk)**2)
    front_nose = raggio1

    file.write(f"Point({center_convergent}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define connecting lines for closed convergent surface
    file.write(f"Line({1}) = {{{new_point_end}, {new_point_start}}};\n")
    file.write(f"Line({2}) = {{{new_point_start}, {lower_conv}}};\n")
    file.write(f"Circle({3}) = {{{center_convergent}, {circle_centre}, {lower_conv}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Circle({4}) = {{{upper_conv}, {circle_centre}, {center_convergent}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Line({5}) = {{{upper_conv}, {upper_point_start}}};\n")
    file.write(f"Line({6}) = {{{upper_point_start}, {upper_point_end}}};\n")




    # Boundary points for the entire domain
    boundary_point_1 = center_convergent + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1
    boundary_point_9 = boundary_point_8 + 1
    boundary_point_10 = boundary_point_9 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H/4}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_bl}, 0, h}};\n") # thick_bl
    file.write(f"Point({boundary_point_9}) = {{{x_start_bl}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_10}) = {{{x_start_bl}, {thick_bl}, 0, h}};\n")


    # Computation of the radius of the circle of the convergent intake B.L., given the center position and two points
    hhh2 = x_start_conv + (x_start_up - x_start_conv)/200      #float(input("Inserisci la coordinata x del centro (h): "))
    kkk2 = (y_upper_conv + y_lower_conv)/2      #float(input("Inserisci la coordinata y del centro (k): "))
    xxx2 = x_start_conv     #float(input("Inserisci la coordinata x del punto sulla circonferenza: "))
    yyy2 = y_upper_conv + thick_bl    #float(input("Inserisci la coordinata y del punto sulla circonferenza: "))
    raggio2 = math.sqrt((xxx2 - hhh2)**2 + (yyy2 - kkk2)**2)
    front_nose_BL = raggio2

    # file.write(f"Point({BL_conv_7}) = {{{x_start_conv + (x_start_up - x_start_conv)/200 - front_nose_BL}, {(y_upper_conv + y_lower_conv)/2}, 0, h}};\n")

    # Define lines for the main boundary
    file.write(f"Line({7}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({8}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({9}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({10}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({11}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({12}) = {{{boundary_point_4},{boundary_point_9} }};\n")
    file.write(f"Line({13}) = {{{boundary_point_9},{boundary_point_1} }};\n")
    file.write(f"Line({14}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({15}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({16}) = {{{boundary_point_7}, {new_point_end}}};\n")
    file.write(f"Line({17}) = {{{boundary_point_5},{boundary_point_10}}};\n")
    file.write(f"Line({18}) = {{{boundary_point_10},{boundary_point_6}}};\n")
    file.write(f"Line({19}) = {{{boundary_point_9},{boundary_point_10}}};\n")


    file.write(f"Transfinite Line{{{12}}} = {400} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{13}}} = {400} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{17}}} = {400} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{18}}} = {400} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{11}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{19}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{14}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{4}}} = {15} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{3}}} = {15} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{7}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{16}}} = {n_bl} Using Progression {1/r};\n")



    file.write("Line Loop(1) = {1, 2, -3, -4, 5, 6, 7, 8, 9, 10, 17, 18, 15, 16};\n")
    file.write("Line Loop(2) = {11, 12, 19, -17};\n")
    file.write("Line Loop(3) = {13, 14, -18, -19};\n")


    #Define the fluid domain
    file.write("Plane Surface(1) = {1};\n") # It tells GMSH to mesh only on 1 and not account for 2
    file.write("Plane Surface(2) = {2};\n")
    file.write("Plane Surface(3) = {3};\n")
    file.write("Transfinite Surface{2} = {")
    file.write(f"{boundary_point_5},{boundary_point_4},{boundary_point_9},{boundary_point_10}")
    file.write("};\n")
    file.write("Transfinite Surface{3} = {")
    file.write(f"{boundary_point_9},{boundary_point_1},{boundary_point_6},{boundary_point_10}")
    file.write("};\n")
    file.write("Recombine Surface{2};\n")
    file.write("Recombine Surface{3};\n")


    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {1, 2, 3};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {8};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {4, 5, 6};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[2].FanPointsList = {5};\n")
    file.write("Field[2].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 2;\n")





    file.write("Physical Surface('VOLUME') = {1:3};\n")


    # Physical lines and surfaces for boundary conditions
    file.write("Physical Line('inlet') = {10, 11};\n")

    file.write("Physical Line('lower_wall') = {12, 13};\n")
    file.write("Physical Line('CONVERGENT') = {1, 2, -3, -4, 5, 6};\n")

    file.write("Physical Line('outlet') = {7, 8, 9};\n")
    file.write("Physical Line('ENGINE') = {14, 15, 16};\n")

















from google.colab import files
files.download("mesh_sbump.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# MACH CALC:
import math

nu =1.78e-5
R =287.7
gamma = 1.4

MACH = 1.6

T_static = 288.15
P_static = 101325
rho = P_static/(R*T_static)
c = math.sqrt(gamma * R * T_static)

vel = c*MACH
Re = rho*vel/nu

T_tot= T_static * (1 + (gamma - 1) / 2 * MACH**2)


P_tot = P_static* (1 + (gamma - 1) / 2 * MACH**2)**(gamma / (gamma - 1))

print(f"T_tot: {T_tot:.3f}, P_tot: {P_tot:.3f}, Re: {Re:.3f}, vel: {vel:.3f}, rho:{rho:.3f} ")

## CALCOLO RECOVERY ##

MACH1=1.17
P_static1 = 0.63
P_tot1 = P_static1* (1 + (gamma - 1) / 2 * MACH1**2)**(gamma / (gamma - 1))
P_ratio = P_tot1/((P_tot/(gamma*MACH*c**2)))

print(f"P_ratio: {P_ratio:.5f}, P_tot1: {P_tot1:.3f},P_tot0: {P_tot/(gamma*MACH*c**2):.3f}")

T_tot: 435.683, P_tot: 430673.234, Re: 37428409.095, vel: 545.084, rho:1.222 
P_ratio: 0.88707, P_tot1: 1.470,P_tot0: 1.657
